In [2]:
from zindi_keyword_spotter.dataset import ZindiAudioDataset
from zindi_keyword_spotter.models import SeResNet3
from zindi_keyword_spotter.lightning import PLClassifier, ZindiDataModule
import pandas as pd
from pathlib import Path
from torchsummary import summary
from torch.utils.data import DataLoader
import pytorch_lightning as pl
import torch
from tqdm import tqdm
import numpy as np
from sklearn.model_selection import train_test_split

In [3]:
pl.seed_everything(14300631)

14300631

In [3]:
data_dir = Path('../data')
df = pd.read_csv(data_dir / 'train.csv')

In [4]:
train_df, val_df = train_test_split(df, stratify=df['label'], test_size=0.3, random_state=42)

In [5]:
train_ds = ZindiAudioDataset(
    54243,
    22050,
    [data_dir / path for path in train_df['fn'].values],
    train_df['label'].values,
)
val_ds = ZindiAudioDataset(
    54243,
    22050,
    [data_dir / path for path in val_df['fn'].values],
    val_df['label'].values,
)

In [6]:
train_loader = DataLoader(train_ds, batch_size=128)
val_loader = DataLoader(val_ds, batch_size=128)

In [7]:
clf = PalSolClassifier(num_classes=len(train_ds.le.classes_), sample_rate=22050)

In [8]:
from pytorch_lightning.loggers import NeptuneLogger

neptune_logger = NeptuneLogger(
    project_name='astromid/zindi-keyword-spotter',
    experiment_name='test',
    params={'max_epochs': 50},
    api_key='eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vdWkubmVwdHVuZS5haSIsImFwaV91cmwiOiJodHRwczovL3VpLm5lcHR1bmUuYWkiLCJhcGlfa2V5IjoiNGJmN2RiYjMtZmNjZS00YmQyLWI3MjItZTk2YzMwMmY3YjI0In0='
)

https://ui.neptune.ai/astromid/zindi-keyword-spotter/e/ZIN-5
NeptuneLogger will work in online mode


In [9]:
trainer = pl.Trainer(gpus=2, max_epochs=50, logger=neptune_logger, distributed_backend='dp')

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0,1]


In [10]:
trainer.fit(clf, train_loader, val_loader)


  | Name  | Type        | Params
--------------------------------------
0 | model | PalSolModel | 935 K 


Saving latest checkpoint..



1

In [5]:
sub = pd.read_csv('../data/raw/SampleSubmission.csv')

In [6]:
sub.head()

,fn,maize streak virus,disease,okukkoola,muwogo,mpeke,mucungwa,greens,garden,mango,...,kasaanyi,suckers,insects,fertilizer,nakavundira,ekiwojjolo,akawuka,ddagala,ebiwojjolo,obutungulu
0,audio_files/00118N3.wav,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,audio_files/00P0NMV.wav,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,audio_files/01QEEZI.wav,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,audio_files/037YAED.wav,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,audio_files/0382N0Y.wav,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
failed_sub = pd.read_csv('../models/seresnet3-11-14_02-08/sub.csv')
failed_sub.head()

,fn,Pump,Spinach,abalimi,afukirira,agriculture,akammwanyi,akamonde,akasaanyi,akatunda,...,suckers,sugarcane,sukumawiki,super grow,sweet potatoes,tomatoes,vegetables,watermelon,weeding,worm
0,audio_files/00118N3.wav,0.000440,0.000526,0.000000e+00,1.000000e-08,0.000002,1.000000e-08,0.000000e+00,0.006462,0.00000,...,0.001057,0.000001,5.900000e-07,2.200000e-07,8.300000e-07,6.100000e-07,0.009993,5.200000e-07,0.002079,0.000078
1,audio_files/00P0NMV.wav,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,1.650000e-06,0.000000,0.00011,...,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,6.580000e-06,0.000000,0.000000
2,audio_files/01QEEZI.wav,0.000000,0.000000,3.000000e-08,0.000000e+00,0.000000,0.000000e+00,4.000000e-08,0.000000,0.00000,...,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000
3,audio_files/037YAED.wav,0.000095,0.000000,0.000000e+00,0.000000e+00,0.000002,0.000000e+00,0.000000e+00,0.000000,0.00000,...,0.006889,0.000000,4.000000e-08,7.300000e-06,0.000000e+00,7.000000e-08,0.000000,0.000000e+00,0.000000,0.000000
4,audio_files/0382N0Y.wav,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000,0.00000,...,0.000000,0.000000,1.100000e-07,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000


In [11]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
le.classes_ = failed_sub.columns.values[1:]

In [13]:
test_ds = ZindiAudioDataset(54243, sub, Path('../data/raw'), 'test')

In [15]:
model = SeResNet3(193, 424, 22050, 128, 2048, 2.0, True, True)

In [16]:
clf = PLClassifier.load_from_checkpoint('../models/seresnet3-11-14_02-08/checkpoints/seresnet3-epoch=86-val_loss=1.648.ckpt', model=model)

In [17]:
clf.eval()
clf.freeze()
probs = []
with torch.no_grad():
    for item in tqdm(test_ds):
        logits = clf(item.unsqueeze(0))
        probs.append(torch.nn.functional.softmax(logits, dim=1).numpy())

100%|██████████| 1017/1017 [00:29<00:00, 34.62it/s]


In [18]:
probs_matrix = np.vstack(probs)
new_sub = pd.DataFrame(probs_matrix)
new_sub.columns = failed_sub.columns.values[1:]
new_sub.insert(0, 'fn', sub['fn'])

In [19]:
new_sub.head()

,fn,Pump,Spinach,abalimi,afukirira,agriculture,akammwanyi,akamonde,akasaanyi,akatunda,...,suckers,sugarcane,sukumawiki,super grow,sweet potatoes,tomatoes,vegetables,watermelon,weeding,worm
0,audio_files/00118N3.wav,5.641948e-03,4.143704e-08,1.400222e-09,1.179730e-06,8.088420e-06,3.408310e-08,1.547708e-10,7.705254e-05,1.130162e-06,...,1.291768e-05,5.844997e-08,1.335717e-08,7.138288e-07,6.160971e-07,2.044306e-06,2.977405e-04,1.721608e-05,6.705738e-03,7.313037e-06
1,audio_files/00P0NMV.wav,6.417027e-20,3.501017e-23,3.919457e-16,6.152171e-15,9.260789e-17,3.018425e-14,6.795860e-07,3.138221e-15,5.374686e-07,...,1.133550e-20,1.312474e-15,2.577132e-21,6.264425e-15,2.288776e-14,3.207303e-11,1.045194e-16,9.441094e-07,1.772470e-16,3.907579e-21
2,audio_files/01QEEZI.wav,3.381641e-17,1.259146e-12,1.132292e-06,1.997551e-12,6.123494e-19,1.466283e-09,1.950575e-06,9.227673e-12,8.084641e-13,...,7.672044e-16,4.293661e-14,2.384056e-13,6.523983e-16,9.866707e-16,1.419034e-12,1.353088e-13,8.709121e-11,7.779886e-11,2.819822e-11
3,audio_files/037YAED.wav,7.842772e-07,9.680712e-25,2.355689e-11,8.499520e-30,2.858377e-11,1.564079e-18,1.014627e-16,1.520115e-11,2.997650e-26,...,3.081443e-03,2.557095e-23,5.287222e-16,2.159115e-10,9.321445e-30,2.092328e-13,7.420534e-25,2.052813e-21,2.221583e-12,2.410583e-18
4,audio_files/0382N0Y.wav,4.885486e-16,7.325424e-13,1.013740e-13,1.040788e-14,1.882770e-15,2.316121e-21,3.754688e-16,8.629038e-21,1.095940e-17,...,1.076778e-18,2.860123e-12,2.028835e-07,2.444872e-16,3.076526e-13,4.583120e-09,2.572578e-14,7.951188e-13,7.615045e-09,1.187632e-17


In [28]:
new_sub.to_csv('../models/seresnet3-11-14_02-08/submission.csv', float_format='%.8f', index=False, header=True)